# Aplicando BERT

**Objetivo**: Treinar uma rede neural BERT no Keras para detectar bots no Twitter utilizando o dataset Twitter-Bot Detection, disponível no Kaggle.

# 1) Setup

A configuração de setup é o processo de preparar e organizar o ambiente para uso. Envolvendo a instalação de bibliotecas e configuração de outros ajustes necessários. O objetivo é criar um ambiente funcional para executar tarefas específicas.

## 1.1) Instalação das bibliotecas

In [10]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## 1.2) Leitura dos Dados

In [2]:
BASE = pd.read_csv('/content/bot_detection_data.csv')

In [3]:
BASE

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention
...,...,...,...,...,...,...,...,...,...,...,...
49995,491196,uberg,Want but put card direction know miss former h...,64,0,9911,True,1,Lake Kimberlyburgh,2023-04-20 11:06:26,teach quality ten education any
49996,739297,jessicamunoz,Provide whole maybe agree church respond most ...,18,5,9900,False,1,Greenbury,2022-10-18 03:57:35,add walk among believe
49997,674475,lynncunningham,Bring different everyone international capital...,43,3,6313,True,1,Deborahfort,2020-07-08 03:54:08,onto admit artist first
49998,167081,richardthompson,Than about single generation itself seek sell ...,45,1,6343,False,0,Stephenside,2022-03-22 12:13:44,star


# 2) Exploração da Base

* **Número de Amostras**: 50000 transações.
* **Número de Features**: 11
* **Tarefa de Classificação**: Prever se é um bot ou não.

## 2.1) Análise das Colunas

In [4]:
BASE.head()

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention


**Explicação das Colunas:**

* User ID: Identificador único para cada usuário no Twitter (não usado diretamente no modelo).
*Username: Nome do usuário no Twitter (pode indicar padrões comuns em bots).
* Tweet: Texto do tweet, principal entrada para o modelo BERT.
* Retweet Count: Número de retweets (pode diferenciar comportamento de bots e humanos).
* Mention Count: Número de menções em um tweet (comportamento de bots pode incluir muitas menções).
* Follower Count: Número de seguidores do usuário (bots podem ter padrões diferentes de humanos).
* Verified: Indica se a conta é verificada (True/False) – contas verificadas são menos propensas a serem bots.
* Bot Label: Rótulo de classificação (0 para humano, 1 para bot); a variável alvo.
* Location: Localização do usuário (pode indicar padrões geográficos).
* Created At: Data de criação da conta (bots podem ter padrões de criação).
* Hashtags: Hashtags usadas no tweet (padrões de hashtags podem indicar bots)

## 2.2) Verificar Valores Ausentes

In [5]:
VALORES_AUSENTES = BASE.isnull().sum()

In [6]:
VALORES_AUSENTES

,0
User ID,0
Username,0
Tweet,0
Retweet Count,0
Mention Count,0
Follower Count,0
Verified,0
Bot Label,0
Location,0
Created At,0


# 3) Pré-Processamento

## 3.1) Função para limpeza de texto

In [11]:
def limparTexto(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#','', text)
    return text

In [12]:
BASE['Tweet'] = BASE['Tweet'].apply(limparTexto)

## 3.2) Tokenizador do BERT

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
def tokenizeTexto(texts, max_length):
    return tokenizer(
        texts.tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
)

# 4) Modelagem

## 4.1) Divisão dos dados

**Ajuste das Colunas**

In [16]:
X = BASE['Tweet']

In [18]:
y = BASE['Bot Label']

**Divisão em treino e teste**

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Tokenização dos Textos**

In [20]:
train_encodings = tokenizeTexto(X_train, 128)

In [21]:
test_encodings = tokenizeTexto(X_test, 128)

**Transformação dos labels em tensores**

In [22]:
y_train = tf.convert_to_tensor(y_train.values)

In [23]:
y_test = tf.convert_to_tensor(y_test.values)

## 4.2) Modelo BERT no Keras

**Importar modelo pré-treinado**

In [24]:
def modeloBert(model_name, max_length):
    bert_model = TFBertModel.from_pretrained(model_name)

    # Construir o modelo em Keras
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
    attention_masks = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")

    bert_output = bert_model(input_ids, attention_mask=attention_masks)[0]
    cls_token = bert_output[:, 0, :]

    output = tf.keras.layers.Dense(1, activation='sigmoid')(cls_token)

    model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output)

    # Compilar o modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

**Criar e compilar o modelo**

In [25]:
model = modeloBert('bert-base-uncased', 128)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

## 4.3) Treinar o modelo

In [ ]:
history = model.fit(
    [train_encodings['input_ids'], train_encodings['attention_mask']],
    y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=32
)

Epoch 1/3


  87/1125 [=>............................] - ETA: 14:16:28 - loss: 0.7013 - accuracy: 0.5036

# 5) Avaliação do modelo

In [ ]:
test_loss, test_accuracy = model.evaluate([test_encodings['input_ids'], test_encodings['attention_mask']], y_test)

In [ ]:
print(f"Teste Acurácia: {test_accuracy}")

In [ ]:
y_pred_probs = model.predict([test_encodings['input_ids'], test_encodings['attention_mask']])

In [ ]:
y_pred = np.round(y_pred_probs).astype(int)

In [ ]:
print(classification_report(y_test, y_pred))

# 6) Persistência e Documentação


## 6.1) Salvar o modelo treinado

In [ ]:
model.save('trained_bert_model.h5')

## 6.2) Documentação

### Análise

A análise tem como objetivo entender os padrões de comportamento entre contas de bots e humanos no Twitter. Para isso, foi realizado um estudo exploratório do dataset contendo informações sobre tweets e usuários. Nesse sentido, o dataset contém colunas como Tweet, Retweet Count, Follower Count, e Verified, que trazem detalhes do conteúdo e características das contas. A coluna Bot Label indica se uma conta é um bot (1) ou humana (0), sendo a variável alvo para classificação.




### Implementação

A implementação desenvolveu uma rede neural BERT em Keras para classificar contas do Twitter como bots ou humanas, baseada nos tweets. O processo incluiu pré-processamento dos dados com limpeza textual e tokenização usando o tokenizador BERT, seguido da construção de um modelo BERT pré-treinado com uma camada densa para classificação. O modelo foi treinado e validado utilizando accuracy para acompanhar o desempenho, e, posteriormente, testado com métricas como accuracy, precision, recall, e F1-score para avaliar sua capacidade de distinguir entre bots e humanos.

### Resultados

As métricas escolhidas para avaliar o modelo de detecção de bots no Twitter foram accuracy, precision, recall e F1-score, cada uma trazendo uma perspectiva única sobre seu desempenho. A accuracy fornece uma visão geral de acertos, mas pode ser enganosa em datasets desbalanceados. A precision é crucial para minimizar falsos positivos (classificar humanos como bots), enquanto o recall foca em capturar todos os bots, evitando falsos negativos. O F1-score, uma média harmônica entre precisão e recall, equilibra ambas as métricas, sendo especialmente útil quando há um trade-off entre erros de classificação. Juntas, essas métricas garantem uma avaliação completa e balanceada do modelo. Os resultados quantitativos serão apresentados após o treinamento do modelo.